In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np 

In [3]:
tokenizer = Tokenizer()

data = open('sonnets.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)


# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [4]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 100)           321100    
_________________________________________________________________
bidirectional (Bidirectional (None, 10, 300)           301200    
_________________________________________________________________
dropout (Dropout)            (None, 10, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 1605)              162105    
_________________________________________________________________
dense_1 (Dense)              (None, 3211)              5156866   
Total params: 6,101,671
Trainable params: 6,101,671
Non-trainable params: 0
______________________________________________

In [5]:
 history = model.fit(predictors, label, epochs=100, verbose=1)

Train on 15462 samples
Epoch 1/100
15462/15462 [==============================] - 48s 3ms/sample - loss: 6.9185 - accuracy: 0.0212
Epoch 2/100
15462/15462 [==============================] - 43s 3ms/sample - loss: 6.5023 - accuracy: 0.0233
Epoch 3/100
15462/15462 [==============================] - 43s 3ms/sample - loss: 6.4076 - accuracy: 0.0239
Epoch 4/100
15462/15462 [==============================] - 44s 3ms/sample - loss: 6.2969 - accuracy: 0.0276
Epoch 5/100
15462/15462 [==============================] - 44s 3ms/sample - loss: 6.2023 - accuracy: 0.0334
Epoch 6/100
15462/15462 [==============================] - 44s 3ms/sample - loss: 6.1174 - accuracy: 0.0376
Epoch 7/100
15462/15462 [==============================] - 45s 3ms/sample - loss: 6.0408 - accuracy: 0.0396s - loss: 6
Epoch 8/100
15462/15462 [==============================] - 45s 3ms/sample - loss: 5.9632 - accuracy: 0.0429
Epoch 9/100
15462/15462 [==============================] - 45s 3ms/sample - loss: 5.8685 - accuracy: 0

15462/15462 [==============================] - 53s 3ms/sample - loss: 1.5204 - accuracy: 0.7406s - loss: 1.5179 - accuracy: 0.74 - ETA: 0s - loss: 1.5195 - accuracy: 0.
Epoch 75/100
15462/15462 [==============================] - 56s 4ms/sample - loss: 1.4954 - accuracy: 0.7419s - loss: 1.4933 - 
Epoch 76/100
15462/15462 [==============================] - 52s 3ms/sample - loss: 1.4852 - accuracy: 0.7423
Epoch 77/100
15462/15462 [==============================] - 50s 3ms/sample - loss: 1.4638 - accuracy: 0.7488
Epoch 78/100
15462/15462 [==============================] - 49s 3ms/sample - loss: 1.4243 - accuracy: 0.7573
Epoch 79/100
15462/15462 [==============================] - 50s 3ms/sample - loss: 1.4094 - accuracy: 0.7589s - los
Epoch 80/100
15462/15462 [==============================] - 54s 4ms/sample - loss: 1.4011 - accuracy: 0.7609
Epoch 81/100
15462/15462 [==============================] - 54s 4ms/sample - loss: 1.3855 - accuracy: 0.7623
Epoch 82/100
15462/15462 [================

In [0]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()

In [0]:
seed_text = "Help me Obi Wan Kenobi, you're my only hope"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)